Bom dia, galera! Então, projeto final de machine learning, task 6:

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results."

então, eu não preciso fazer 'nada' na linha abaixo? Digo, não preciso programar? Eu deixo a seguinte linha nessa task: 

In [ ]:
dump_classifier_and_data(clf, my_dataset, features_list)"
Daí o programa roda certinho (o poi_id.py), mas quando rodo o programa tester.py, dá o seguinte erro:
"Traceback (most recent call last):
 File "tester.py", line 109, in <module>
   main()
 File "tester.py", line 106, in main
   test_classifier(clf, dataset, feature_list)
 File "tester.py", line 37, in test_classifier
   for train_idx, test_idx in cv:
TypeError: 'StratifiedShuffleSplit' object is not iterable


Esse problema acontece porque vc está usando uma versão "muito atualizada" do Scikit Learn.

Após a versão 0.19.1 o `StratifiedShuffleSplit` não pode mais ser "iterado".

Há duas soluções:

1) Vc fazer um _downgrade_ do Scikit Learn, e;
2) Alterar o `tester.py`.

Sugiro que faça o _downgrade_ para não alterar o `tester.py`.

Por favor, acompanhe os números das linhas do repositório para nos situarmos.

https://github.com/udacity/ud120-projects/blob/master/final_project/tester.py

A primeira coisa é a mudança do nome do módulo:

In [ ]:
# Linha 15
# De:
from sklearn.cross_validation import StratifiedShuffleSplit

#Para:
from sklearn.model_selection import StratifiedShuffleSplit

Falta inserir depois da linha 28 uma nova linha.

In [ ]:
# Exemplo de como proceder.

# Isso é um análogo da linha 28.
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)

# Isso é o que falta.
sss.get_n_splits(X, y)

Da linha 33 até a 43 vc terá que substituir por:

In [ ]:
# Exemplo genérico que deve ser substituído com os valores reais.
for train_index, test_index in sss.split(X, y):
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

Observe que eu usei o próprio exemplo do Scikit Learn. Logo, vc terá que substituir os `x_train`, `x_test`, `y_train` e `y_test` pelo `features_train`, `features_test`, `labels_train` e `labels_test`.

O detalhe que ficou obscuro.

In [ ]:
X = features
y = labels

para mim não funcionou, vou mandar como eu fiz:

In [ ]:
def test_classifier(clf, dataset, feature_list, folds = 1000):
   data = featureFormat(dataset, feature_list, sort_keys = True)
   labels, features = targetFeatureSplit(data)
#    cv = StratifiedShuffleSplit(labels, folds, random_state = 42).get_n_splits(features,labels)
#    sss = StratifiedShuffleSplit(labels, folds, random_state = 42)
   sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
   sss.get_n_splits(features,labels)
   #.get_n_splits(features,labels)
   true_negatives = 0
   false_negatives = 0
   true_positives = 0
   false_positives = 0
   for train_index, test_index in sss.split(features, labels):
       features_train, features_test = features[train_index], features[test_index]
       labels_train, labels_test = labels[train_index], labels[test_index]

dá esse erro: "Traceback (most recent call last):
 File "tester.py", line 120, in <module>
   main()
 File "tester.py", line 117, in main
   test_classifier(clf, dataset, feature_list)
 File "tester.py", line 55, in test_classifier
   features_train, features_test = features[train_index], features[test_index]
TypeError: only integer scalar arrays can be converted to a scalar index

Creio que vc não entendeu o que foi explicado, mas vc fez justamente da maneira como era para ser feito.
1) Cria-se o objeto `sss`;
2) Usa-se o `get_n_splits` nesse objeto criado;
3) Por fim, aplica o `split` no `sss`.
É a aplicação do que está na documentação do Scikit Learn.

Contudo, atente-se ao fato de vc ter configurado valores diferentes para `n_split`, `random_state`, etc daquele do original.

In [ ]:
# Original
cv = StratifiedShuffleSplit(labels, folds, random_state = 42)

# Ajustado
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)

usando o original dá o seguinte erro: "ValueError: test_size=1000 should be smaller than the number of samples 141
". é o folds que na função o default está em 1000

vc está interpretando incorretamente o `test_size`. Ele deve ser um valor entre 0 e 1 (0 e 100%). Além disso, quando o `test_size` é omitido o _Scikit Learn_ entende que será usado o valor default que é 10% ou 0,1. Já o `n_split` será o "folds", OK? Para dirimir a sua dúvida, por favor, leia a Documentação.

test_size : float, int, None, optional
If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. If int, represents the absolute number of test samples. If None, the value is set to the complement of the train size. By default, the value is set to 0.1. The default will change in version 0.21. It will remain 0.1 only if train_size is unspecified, otherwise it will complement the specified train_size.

n_splits : int, default 10
Number of re-shuffling & splitting iterations

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.htm

ainda não entendo o meu erro, eu coloco como na documentação: 

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size = 0.5, random_state = 42)

o for fica assim: (e dá o erro)

In [ ]:
for train_index, test_index in sss.split(features, labels):

File "tester.py", line 120, in <module>
   main()
 File "tester.py", line 117, in main
   test_classifier(clf, dataset, feature_list)
 File "tester.py", line 55, in test_classifier
   features_train, features_test = features[train_index], features[test_index]
TypeError: only integer scalar arrays can be converted to a scalar index

de fato, train_index não é inteiro

sss.split(features, labels) aparece como sendo um generator

a primeira saída de train_index para mim é: 

In [ ]:
[138  82   5  18  89  66  12 133  90  65  27 125  22  64  32  13 115 110
 83  25 109 127  86 116  20  77  29  80  81  62  10   8   0 134  36  47
 52 107  51  72  41  50 137  43  31  78  34  40  79  97 103 102  93  57
 74  91  17  46  11  39  49  30 106  37 135 104   4  21  55 117]

Bom, parece que ele está gerando os "índices" corretamente para fazer o _subset_ de `features` e `labels`

Faça um teste, converta o `features` e o `labels` para numpy array (`np.array()`).
Se vc fizer isso vai dar certo.

funcionou! mas está demorando para rodar! era para ser rápido?

acho que eu tb posso estar errando em como exportei meu clf no poi_id.py

Demora viu. Se vc optou por usar o AdaBoost, demorará alguns minutinhos para rodar.

Se vc está com dúvida se o modelo travou, configure o `verbose` para 5, assim a cada etapa o modelo irá imprimir algo e vc terá uma noção se travou ou não

só para confirmar, eu coloquei 

In [ ]:
clf= grid_search.best_estimator_

Ele vai retornar o modelo que obteve os melhores resultados. Daí pode ser o modelo que obteve a melhor média de score do test dataset, isso dependerá de como vc configurou.

In [ ]:
class sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=None, iid='warn', refit=True, cv='warn', verbose=0, pre_dispatch='2*n_jobs', error_score='raise-deprecating', return_train_score='warn')[source]

verbose : integer
Controls the verbosity: the higher, the more messages.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

ah ok! queria só saber se estava certo a maneira de obter o clf do melhor modelo a partir de grid_search

Vc pode dar uma olhada em todo o resultado do `GridSearchCV` usando o atributo `cv_results_`

In [ ]:
# Retorna um dicionário com os valores obtidos do `GridSearchCV`.
clf.cv_results_

Além disso, vc pode transformá-lo num pandas para facilitar a visualização.

massa! obrigado :slightly_smiling_face: agora deu tudo certo :slightly_smiling_face: !!!! É que era  pesado o que eu tava usando antes, eu coloquei agora so um gaussian naive bayes e deu o seguinte ao rodar tester.py:

In [ ]:
Pipeline(memory=None,
    steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
 svd_solver='auto', tol=0.0, whiten=False)), ('gnb', GaussianNB(priors=None, var_smoothing=1e-09))])

In [ ]:
Accuracy: 0.84789    Precision: 0.40426    Recall: 0.42222    F1: 0.41304    F2: 0.41850
    Total predictions:  355    True positives:   19    False positives:   28    False negatives:   26    True negatives:  282

antes eu tava rodando, para testar,  o suport vector classifier: e colocava como parametros:

In [ ]:
svc__gamma' : [1,5,10],
   'svc__kernel' : ['linear','rbf']

acho que isso era muito pesado!

Não se esqueça que o SVM necessita que vc faça o _feature scale_, caso contrário ele vai demorar séculos

o codigo funcionou agora com o SVC, não demora mais séculos hahahahaa é que eu fazia o MinMaxScaler() na "mao", o certo é no Pipeline. Eu fazia depois de chamar o Pipeline, daí não adianta

agora vem a seguinte coisa:

In [ ]:
Got a divide by zero when trying out: Pipeline(memory=None,
    steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
 svd_solver='auto', tol=0.0, whiten=False)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
 decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
 kernel='rbf', max_iter=-1, probability=False, random_state=None,
 shrinking=True, tol=0.001, verbose=False))])
Precision or recall may be undefined due to a lack of true positive predicitons.

só que olhe como são as minhas predições usando SVC:

In [ ]:
prediction [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Não há problema algum vc aplicar o _feature scale_ mesmo "não sendo necessário". Eu faço isso como um preparativo para usar os algoritmos.

Isso vai dar problema na hora de calcular as métricas, pois não tem nenhum label do tipo 1.

entendi! desconfiava mesmo que fosse esse monte de zeros!

Vc está usando o `StratifiedShuffleSplit`?

estou

no tester.py

Como está o seu `GridSearchCV`?

está assim:

In [ ]:
grid_search = GridSearchCV(pipeline, parameters, cv=5,n_jobs=-1, verbose=5)

Observe que vc necessita fazer algum tipo de _cross_validation_, pode ser um "manual" vc criando os datasets de treino e testes ou vc usando o built-in `cv`.

Vc está usando um `KFoldsStratified` que é o _default_ do `GridSearchCV`.

ah então, eu não entendi bem o que é esse cross-validation

O _cross_validation_ é uma maneira de vc avaliar o seu modelo se ele não vai ficar _overfitted_ ou incorrer em outros tipos de problemas.

No caso vc usou `cv = 5`, isto quer dizer que vc usará 5 folds, logo haverá 5 simulações alternando o fold usado como test. No final vc fará uma média dos resultados das métricas. Isso é o que vc está fazendo.

estou lendo sobre isso! realmente, eu estava fazendo às cegas, não tinha captado a mensagem do cross-validation! estou com 5 folds. Então, eu deveria alterar esse numero?

Observe que o fold significa que vc irá dividir o seu dataset. Vamos fazer contas.
1) Vc deve ter 140 observações aproximadamente;
2) Caso vc adote `cv = 10`, vc terá  10% de teste sobrará 126 observações para treinar e 14 para teste;
3) A proporção de POI e non-POI é de 18/122 (mais ou menos);
4) O dataset de teste (14 observações) deverá seguir a mesma proporção 18/122.
Logo, se vc opta por muitos `cv` o seu test dataset pode ficar extremamente reduzido e o pior é que ele não conseguirá manter a proporcionalidade. No limite, `cv` pode ser igual ao número de observações.

Posto isto, para datasets pequenos aconselha-se o uso do `StratifiedShuffleSplit`, conforme apontado nas instruções do projeto.

ah entendi!

então, eu deveria mudar, por exemplo:

In [ ]:
from  sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = \
   train_test_split(features, labels, test_size=0.3, random_state=42)

de train_test_split para StratifiedShuffleSplit?

como a gente fez no tester.py?

Não é necessário. Pode ser mais simples.

Diferentemente do `tester.py` onde criamos tudo passo-a-passo. No GridSearchCV vc pode apenas criar o objeto e definir como sendo o `cv`.

In [ ]:
# Criando o objeto StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)

# No seu GridSearchCV
objeto = GridSearchCV(XX, XX, XX, cv = sss)

Vc pode tanto definir um número para o `cv` quanto um objeto do tipo SSS. O `GridSearchCV` é sagaz e entende o que queremos fazer.

Todavia, atente-se ao fato de que vc terá que treinar o seu `objeto` usando todo o `features` e `labels`, pois *internamente* o `GridSearchCV` fará para vc o manejo do que será dataset de treino e de testes.

Isso pode automatizar demasiadamente e causar confusão, pois parece que estamos esquecendo de alguma coisa.

não entendi essa sua ultima parte. eu faço o cv=sss e dá um erro: ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.
o que sera que esta havendo?

Que estranho!

Vc só precisa preencher os `...`

In [ ]:
# Normalmente eu fazia asssim.
objeto = GridSearchCV( ... , cv = StratifiedShuffleSplit(n_splits=20, random_state=42))

eu fiz assim:

In [ ]:
sss = StratifiedShuffleSplit(n_splits=18, test_size=0.1, random_state=0)

grid_search = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=5,cv=sss)

e o problema está no fit:

In [ ]:
grid_search.fit(features_train, labels_train)

Vc só treina.

Depois vc usa o `cv_results_`

In [ ]:
# Para visualizar os resultados.
clf.cv_results_

mas da erro bem no fit

não sei se ajuda, mas olha o erro todo:

In [ ]:
ValueError: The least populated class in y has only 1 member, which is too few. 
The minimum number of groups for any class cannot be less than 2.

Seus labels provavelmente estão com algum erro

Vc tem que usar o features e labels completos, sem separar em treino e test.

Imprima para que possamos ver o conteúdo dos labels, por favor.

In [ ]:
[array([400000.]), array([350000.]), array([750000.]), array([0.]), array([600000.]), array([600000.]), array([350000.]), array([700000.]), array([0.]), array([600000.]), array([600000.]), array([325000.]), array([0.]), array([800000.]), array([1000000.]), array([1200000.]), array([5249999.]), array([1350000.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([425000.]), array([0.]), array([0.]), array([0.]), array([1000000.]), array([1000000.]), array([800000.]), array([900000.]), array([0.]), array([0.]), array([2500000.]), array([0.]), array([509870.]), array([800000.]), array([2000000.]), array([8000000.]), array([900000.]), array([800000.]), array([5600000.]), array([0.]), array([70000.]), array([0.]), array([600000.]), array([0.]), array([0.]), array([1500000.]), array([7000000.]), array([0.]), array([0.]), array([600000.]), array([1200000.]), array([0.]), array([0.]), array([325000.]), array([0.]), array([3100000.]), array([0.]), array([0.]), array([850000.]), array([1700000.]), array([0.]), array([0.]), array([700000.]), array([650000.]), array([0.]), array([4175000.]), array([400000.]), array([1100000.]), array([1300000.]), array([300000.]), array([500000.]), array([250000.]), array([700000.]), array([1750000.]), array([300000.]), array([250000.]), array([1000000.]), array([0.]), array([0.]), array([0.]), array([3000000.]), array([300000.]), array([1150000.]), array([500000.]), array([700000.]), array([700000.]), array([1100000.]), array([0.]), array([0.]), array([200000.]), array([1000000.]), array([2600000.]), array([0.]), array([750000.]), array([400000.]), array([1000000.]), array([0.]), array([0.]), array([0.]), array([0.]), array([100000.]), array([0.]), array([2000000.]), array([97343619.]), array([0.]), array([750000.]), array([0.]), array([200000.]), array([0.]), array([1500000.]), array([0.]), array([400000.]), array([1250000.]), array([0.])] labels

Complementando o Anderson, o StratifiedSuffleSplit já faz as divisões entre teste e treino, quando você não adiciona o parâmetro `test_size=x` ele assume que 50% é para treino e 50% para teste

Seu labels realmente estão com erro

Vc está dando scale nos labels?

Os labels deveriam ser apenas `True` e `False`

Sim, tô achando que o Vagner deu _scale_ em tudo.

Mas pelos valores acredito que não, Vagner você colocou o `'poi'` como primeiro valor na feature_list?

é, tá com erro nos labels! coloquei poi sim

o incrível é que tava rodando

O poi tem de ficar em primeiro sempre:

In [ ]:
features_list = ['poi', 'salary', 'deferral_payments', 'total_payments', ...]

Bom, problema identificado!

O algoritmo é um moedor de carne. Ele foi feito para rodar! Se as entradas são corretas ou não não importa para ele, pois fará o máximo para te dar um resultado que não seja "erro".

na task 1, a feature selection estava retornando:

não tava retornando poi

é aí o erro né?

In [ ]:
['bonus', 'exercised_stock_options', 'shared_receipt_with_poi', 'director_fees', 'deferred_income']

Sim você tem de modificar sua features_list e manter o poi no inicio e adicionar as demais

consertei! agora meu labels deram isso:

In [ ]:
[array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([1.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([1.]), array([0.]), array([0.])]

 isso ae
 
 0 é nonPOI e 1 é POI

agora eu tenho como resultado ao roda poi_id.py:
    
mas o tester.py ainda dá o erro lá da divisão por zeros (predictions ainda continua 0). Mas estranho o recall e precision terem aumentado, não?

In [ ]:
prediction [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
recall 0.8611111111111112 precision 0.8611111111111112

você esta usando python 2.7 ou python 3?

2.7. mas acho que o erro está em outra parte pq agora o codigo foi alterado ne, eu tenho que consertar outras partes

Isso, verifique se o dicionário salvo esta correto e se a lista das features também

Esse problema da divisão por zero é complicado.

Isso ocorre pelo dataset conter muitos zeros, alguns testes vão fazer a divisão usando esse zero e acabar dando esse erro

mas isso depende do resultado do seu modelo tbm, se ele só consegue prever zeros, vai dar esse probleminha.

acho que o problema era a formulação do SVC! pois com GNB deu resultados bem razoaveis! E com AdaBoost da certinho no tester.py, com recall e precision maior do que 0.3.

aqui no meu codigo, AdaBoost e KNN não rodam com PCA. É isso mesmo? Se sim, há algum motivo?

esquece, era erro meu

não entendi porque tem o tester.py. Nele é calculado, na "mão" precision e recall, certo? Mas isso pode ser calculado no poi_id.py através de:

não é? E para mim os resultados dão diferentes, bem diferentes!

In [ ]:
rec = recall_score(predict,labels_test, average='micro')
prec = precision_score(predict,labels_test, average='micro')

É devido a quantidade de vezes que você roda, lá é uma media geral de 1000 iterações, você fazendo 1000 iterações e mantendo o StratfiedShuffleSplit com random_state=42 deve gerar um valor bem proximo

ah então o valor deve ser bem proximo? Estranho que para mim tá dando bem diferente

No poi_id.py eu escrevo assim:

In [ ]:
rec = recall_score(predict,labels_test, average='micro')
prec = precision_score(predict,labels_test, average='micro')

e o clf para a task 6 como:

In [ ]:
clf= grid_search.best_estimator_

Creio que o seu problema seja a configuração do `recall_score`. Entretando, devo ressaltar que isso em termos gerais é um pouco temerário quando acontece, pois se nota que o modelo pode estar pouco "estável". Um dos motivos para acontecer isso é o _Data Leakage_. Pode ser até mesmo pouquíssimas simulações no cross_validation, pois isso é um resultado de uma média.

Observe ao fato de vc ter definido `average = 'micro'`. Creio que devemos explanar sobre isso:

* `macro`: Retorna apenas 1 valor que é a média do recall de cada label (no caso, label zero e label um);
* `micro`: Também retorna apenas 1 valor, mas usa um conceito de calcular o total de True Positive e o Total de False Positive para calcular o recall. Observe que no projeto temos label zero e label um, cada label tem o seu recall, precision e accurácia.
* `None`: Retorna *dois* valores, o primeiro é o recall do label zero e o segundo do label um.

ah tah, cada label, 0 ou 1, tem o seu recall. logo, seria o mais correto usar a opcao 'None'

supondo que o problema de compatibilidade entre os recalls seja por Data Leakage, como eu faria para identificar qual feature esta com muitos NaNs? Força bruta mesmo, contando?

Tem uma outra opção quando vc omite o `Average`, daí ele só retorna o recall do label 1. Daí fica a teu critério qual deles usar, mas o modelo usado no `tester.py` está alinhado com o `None` ou quando vc omite o `average`.

Vc pode contar, vai ter variável aí com mais de 90% de `NaN`'s, sugiro que vc evite de usá-lo.

---

ola galera, boa noite! Então, no projeto final de machine learniing, mexendo no grid_search_cv:

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5,test_size = 0.5, random_state=42)

scoring = {'accuracy':make_scorer(accuracy_score),
        'precision':make_scorer(precision_score),
        'recall':make_scorer(recall_score)}


grid_search = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=5,cv=sss,scoring=scoring)

quando eu coloco scoring para o GridSearchCV lidar com as metricas, accuracy, etc, o que esta'faltando? Pq da o seguinte erro:

In [ ]:
ValueError: For multi-metric scoring, the parameter refit must be set to a scorer key to refit an estimator with the 
best parameter setting on the whole data and make the best_* attributes available for that metric. If this is not 
needed, refit should be set to False explicitly.'AUC' was passed

eu coloquei:

agora funciona!

mas ainda o meu recall do tester.py da diferente do recall que eu calculo no poi_id.py!

In [ ]:
grid_search = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=5,cv=sss,scoring=scoring,refit='recall',
                           return_train_score=True)

Como vc está usando o `make_scorer` o `GridSearchCV` precisa de uma métrica que vc julga como favorita, pois assim ele conseguirá retornar um melhor estimador quando o `best_estimator_` for usado.

A sua preferência será definida no `refit`. Portanto, defina o refit, por exemplo, como `refit = 'recall'`.

Faça esse teste e veja se resolve o seu problema.

---

hein galera, boa noite! Agora eu queria que voces me explicassem uma coisa, eu sei que é fora do escopo do curso, mas se alguém puder me elucidar qual a diferença entre: "Decision Tree", "Random Forest" e "AdaBoost", eu agradeceria. É que no projeto final eu gostaria de usar random forest e adaboost, mas não ficou claro para mim cada um deles. Eu sei que eles tem alguma coisa em comum, mas não entendi bem o que é! Vlw

Perdoe-me pela demora em responder.

Há algoritmos que compartilham um pouco da sua essência. Random Forest e Decision Trees são bem parecidos, pois o Random Forest foi criado baseado na Decision Trees.

**Decision Trees**

A partir de questões simples cria-se uma árvore de classificação, a sequência das "perguntas" é de acordo com o índice de impureza (que pode ser calculado a partir do Gini, por exemplo).

**Random Forest**

É a aplicação da Decision Trees de forma sistemática (centenas de vezes), mas usando o conceito de _Bootstrapping_ para criar diferentes amostras e consequentemente diferentes árvores de decisão. Observe que nesse algoritmo há a "randomização", isto é, nem sempre os resultados serão iguais (vc deverá o configurar o `random_seed`), pois foi feito uma escolha aleatória das amostras (no _bootstrapping_).

Portanto, o resultado do algoritmo será baseado em várias árvores e apenas não só por uma (que é o caso do Decision Trees). Há outros detalhes que vou omitir agora, mas caso vc sinta necessidade posso te explicar para um entendimento mais profundo.

**AdaBoost**

Assim como o Random Forest o AdaBoost tbm é baseado na Decision Trees. A diferença está em:

* Será criado uma "Floresta de Galhos", isto é, não será feito uma árvore completa como é o caso do Random Forest. A partir desses galhos cria-se uma árvore de classificação;
* Cada galho terá um peso diferente no algoritmo, diferentemente do caso do Random Forest em que todas as árvores possuíam o mesmo peso;
* A sequência dos galhos para criar a Árvore de Classificação é muito importante, pois pode alterar completamente o resultado final. Note que o AdaBoost usa o conceito de propagação de erro (a partir do `sample_weight`), que é calculado com base nas classificações erradas que ele fez;
* Por fim, o _bootstrapping_ é usado para criar os galhos (análogo ao Random Forest). Note que o `sample_weight` é usado nesta etapa para dar mais ênfase nas observações que foram incorretamente classificadas.

Bom, tá aí um resumão, só não sei se ficou fácil de entender.

Ressalto que o AdaBoost é bem mais complicado que os demais, pois é uma "evolução" da Decision Trees.

---

ola galera, bom dia! Então, projeto final de machine learning! O meu recall calculado com grid_search dá super bom nos dados de teste, mas quando eu rodo o tester.py dá um valor bem baixo. Alguém poderia me dar uma mao? Não entendendo essa discrepância! Eu tive que alterar o tester.py para usar o StratifiedShuffleSplit do model selection, mas não vejo erros, uma vez também que o @AH Uyekita me ajudou nisso. Por que estaria o recall dando baixo no tester.py? Alguém mais já obteve isso?

Creio que seja o probleminha do `Average = 'micro'`. Sugiro que vc faça o cálculo na mão imprimindo a _Confusion Matrix_, isso será muito revelador.

Colocando a opção None, as coisas melhoraram mas quando eu coloco o seguinte:

In [ ]:
scoring = {'accuracy':make_scorer(accuracy_score),
        'precision':make_scorer(precision_score,average=None),
        'recall':make_scorer(recall_score,average=None)}
"
da o seguinte erro ao rodar o grid search:
"
ValueError: scoring must return a number, got [0.86440678 0.        ] (<type 'numpy.ndarray'>) instead. (scorer=precision)

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5,test_size = 0.5, random_state=42)

scoring = {'accuracy':make_scorer(accuracy_score),
        'precision':make_scorer(precision_score,average=None),
        'recall':make_scorer(recall_score,average=None)}


grid_search = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1,cv=sss,scoring=scoring,refit='recall', 
                           return_train_score=True)

todo o projeto está baseado no _recall_ e _precision_ do label um. Talvez seja melhor vc omitir o `average` e só trabalhar com as métricas do label um. Aparentemente o problema é que retornou dois valores ao invés de um, o que é esperado pelo `GridSearchCV`.

se eu omitir o average fica como estava anteriormente. não bate o recall com o do tester.py

Então, tenta isso:

In [ ]:
# Quero somente o segundo elemento da lista que é o recall do label 1.
make_scorer(precision_score,average=None)[1]

não dá certo :disappointed: não suporta índice
TypeError: '_PredictScorer' object does not support indexing

Isso está correto.

In [ ]:
scoring = {'accuracy':make_scorer(accuracy_score),
        'precision':make_scorer(precision_score),
        'recall':make_scorer(recall_score)}

Se o seu problema é que os valores obtidos durante o processo de "modelagem/calibragem" diferem em muito do `tester.py`.
0) Dado que não há problemas de código;
1) Verificar as suas variáveis;
2) Ajustar os parâmetros;
3) Analisar se faz sentido o que vc está calculando com base nas informações obtidas.
Note que o processo de criação de um modelo é "iterativo" e provavelmente vc terá que repetir alguns passos até atingir o seu melhor.

Digamos que vc atingiu a sua primeira iteração, agora basta vc revisar as suas variáveis e consequentemente os parâmetros. Levante perguntas para o seu modelo.
1) Estou fazendo o _cross_validation_ corretamente???
2) As minhas variáveis são boas!?
Infelizmente, esse projeto não é "linear" e requer que o aluno revisite perguntas já respondidas (as Tasks 1 ao 6) para buscar alguma falha ou erro.

os labels dos dados no tester e no poi_id.py são diferntes

Não, são iguais. Há apenas dois labels 0 e 1.

Vc não está usando o `MinMaxScaler` nos labels né!?

Outra coisa... Qual é a composição da sua lista `features_list`?

eu me expressei errado! quando dou len(labels) no poi_id.y da 140, ja no tester.py da 141

não, não faço scaling nos labels

Bom, não vejo como isso poderia travar o seu `test_classifier`, mas a priori deveriam ser iguais. Por acaso vc está atualizando o `dict`?

usando o dict onde?

Repare que a função `test_classifier` tem como input um `dict`. Esse `dict` é exportado pelo `dump_classifier_and_data`. No Task 3 vc tem que criar o `my_dataset` que será exportado pelo `dump_classifier_and_data` e este por sua vez será usado pelo `test_classifier`.

ah então ta aí! Eu so criava novos features, mas não a partir de um novo data_dict, modifica na mão o features

fazia assim ó:

é basicamente, fora o 'poi' e 'bonus', definir um novo feature como feature[i]/'bonus'[i]

In [ ]:
def relative_to_bonus(features):
    """
        This function gives the quocient of the best features
        relatively to the "bonus" feature.
        Input(s):
                 -features: features of the data
        Output(s):
                  -new_features: features of the data divided by bonus
        Remarks: We do not normalize the first column, which is "bonus".
        To avoid DivisionByZero error, we replace zeros by the mean!        
    """
    
    
    new_features = []
#    features
    # calculating the average of the first feature!
    avg_0=0
    for i in range(len(features)):
        avg_0+=features[i][1]
    
    # iterating through features
    for i in range(len(features)):
        feat_j=[]
        for j in range(5):
#            if the the list element is not zero!
            if(features[i][1]!=0):
                if(j!=1 and j!=0):
                    feat_j.append(features[i][j]/features[i][1])
                elif(j==0):
                    feat_j.append(features[i][j])
                else:
                    feat_j.append(features[i][1])
#            if there is a zero, replace it by the average!
            if(features[i][1]==0):
                if(j!=1 and j!=0):
                    feat_j.append(features[i][j]/avg_0)
                elif(j==0):
                    feat_j.append(features[i][j])
                else:
                    feat_j.append(avg_0)
                    
        new_features.append(np.array(feat_j))
        
    return new_features

OK! Então não se esqueça de criar o `my_dataset` que deve possuir a nova _feature_.

muito obrigado :slightly_smiling_face: agora vou seguir batalhando aqui :slightly_smiling_face:

Além disso, o `my_dataset` não deve conter os _outliers_.

Pense que o `my_dataset` é o seu banco de dados completamente limpo e com as novas variáveis já inclusas.

bem que eu estava achando estranho não ter que exportar, ou algo assim, os dados e features criados heheeheh

consegui!!! recall de 1 e precision de 1. Nada mal!

só tome cuidado com valores muito bons!!! Isso pode ser um alerta para o _Overfitting_!!!

certo! mas esse valor é nos dados de teste mesmo

Blz!! Tá muito bom os resultados. Isso é usando o `test_classifier`?

Note que a nota será feita a partir do `test_classifier`. Portanto, sugiro que faça várias simulações caso vc tenha optado pelo AdaBoost ou Random Forest para ver se os resultados de cada "rodada" do `test_classifier` fiquem próximos.

ficaram bem proximas! :slightly_smiling_face: recall e precision de 1 é o resultado do test_classifier

Ahhhh muleke!! É 10 então! hahahahha

vlw! ufa! ralei peito para fazer isso! Estou muito orgulhoso do que já fiz. Po, complicado mesmo esse projeto. Um cara que não tem muito jeito para programar, não faz isso nem lascando. Botando fé na Udacity